###Creating your DBFS mount for data storage
1. Create ADLS Gen2 Storage account
2. Create storage container in your storage account
3. Create Azure service principal and secret
4. Grant access to service proncipal for storage account
5. Mount storage container

####1. Create ADLS Gen2 Storage account
* Click "Create a resource" on your Azure portal home page
* Search for "Storage account" and click the create button
* Create a storage account using the following
    * Choose an appropriate subscription
    * Select an existing or Create a new Resource group
    * Choose a unique storage account name Ex: (mystorageaccount)
    * Choose a region (Choose the same region where your Databricks service is created)
    * Select performance tier (Standard tier is good enough for learning)
    * Choose storage redundency (LRS is good enough for learning)
    * Click Advanced button to move to the next step
    * Select "Enable hierarchical namespace" on the Advanced tab
    * Click "Review" button
    * Click the "Create" button after reviewing your settings


####2. Create storage container in your storage account
* Go to your Azure storage account page
* Select "Containers" from the left side menu
* Click "+ Container" button from the top menu
* Give a name to your containe (Ex dbfs-container)
* Click the "Create" button

####3. Create Azure service principal and secret
* Go to Azure Active Directory Service page in your Azure account (Azure Active Directory is now Microsoft Entra ID)
* Select "App registrations" from the left side menu
* Click (+ New registration) from the top menu
* Give a name to your service principal (Ex databricks-app-principal)
* Click the "Register" button
* Service principal will be created and details will be shown on the service principal page
* Copy "Application (client) ID" and "Directory (tenant) ID" values. You will need them later
* Choose "Certificates & secrets" from the left menu
* Click "+ New client secret" on the secrets page
* Enter a description (Ex databricks-app-principal-secret)
* Select an expiry (Ex 3 Months)
* Click the "Add" button
* Secret will be created and shown on the page
* Copy the Secret value. You will need it later



####4. Grant access to service proncipal for storage account
* Go to your storage account page
* Click "Access control (IAM)" from the left menu
* Click the "+ Add" button and choose "Add role assignment"
* Search for "Storage Blob Data Contributor" role and select it
* Click "Next" button
* Click the "+ Select members"
* Search for your Databricks service principal (Ex databricks-app-principal) and select it
* Clcik "Select" button
* Click "Review + assign" button twice

####5. Mount storage container

#####5.1 Define necessory variables

# IMPORTANT: Protect your sensitive access data and variables!:

In [0]:
import os

In [0]:
storage_account_name = os.getenv("STORAGE_ACCOUNT_NAME")
container_name = "api-pipeline"
mount_point = "files"
client_id = os.getenv("CLIENT_ID")
tenant_id = os.getenv("TENANT_ID")
client_secret = os.getenv("CLIENT_SECRET_VALUE")

#####5.2 Define mount configs
You can follow the instruction and code sample from below documentation page

https://learn.microsoft.com/en-us/azure/databricks/dbfs/mounts#--mount-adls-gen2-or-blob-storage-with-abfs

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
        "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
        "fs.azure.account.oauth2.client.id": f"{client_id}",
        "fs.azure.account.oauth2.client.secret": f"{client_secret}",
        "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

#####5.3 Mount the container

In [0]:
dbutils.fs.mount(
  source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
  mount_point = f"/mnt/{mount_point}",
  extra_configs = configs)

#####5.4. List contents of your mount point

In [0]:
%fs
ls /mnt/files/

#####5.5. Upload your sample data folder to your mounted location

#####5.6. List contents of your mount point

In [0]:
%fs
ls /mnt/files

#####5.7. Unmount /mnt/files directory

In [0]:
# if you want to unmount your directory
# %fs
# unmount /mnt/files